# Rydberg Atom Array Simulation with Maestro

This notebook demonstrates how to simulate a 1D Rydberg atom array using Maestro's Matrix Product State (MPS) backend. We will:

1. **Build a Trotterized circuit** for adiabatic state preparation
2. **Compute a phase diagram** by sweeping detuning and Rabi frequency
3. **Measure spatial correlations** using MPS bitstring sampling

---

## Physics Background

Rydberg atom arrays are a leading platform for quantum simulation. Neutral atoms trapped in optical tweezers interact via strong van der Waals interactions when excited to Rydberg states. The system is governed by:

$$H = \frac{\Omega}{2} \sum_i X_i - \Delta \sum_i n_i + V \sum_{\langle i,j \rangle} n_i n_j$$

where:
- **$\Omega$** (Rabi frequency) drives transitions between ground and Rydberg states
- **$\Delta$** (detuning) controls the energy cost of excitation
- **$V$** (interaction) implements the Rydberg blockade — preventing adjacent atoms from both being excited
- **$n_i = (I - Z_i)/2$** is the Rydberg excitation number operator

By adiabatically ramping $\Delta$ from negative to positive, the system undergoes a quantum phase transition into a **Z2-ordered phase** — an alternating pattern $|1010\ldots\rangle$.

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

import maestro
from maestro.circuits import QuantumCircuit

%matplotlib inline
plt.rcParams['figure.dpi'] = 120

## Step 1: Building the Rydberg Circuit

We use first-order Trotterization with a linear ramp schedule:
- $\Omega$ ramps from $0 \to \Omega_{\text{final}}$
- $\Delta$ ramps from $-5.0 \to \Delta_{\text{final}}$

Each Trotter step implements:
1. **ZZ interactions** (Rydberg blockade) via CX-Rz-CX on nearest-neighbor pairs
2. **Single-qubit Rx** (drive) and **Rz** (detuning + interaction corrections)

In [ ]:
def create_rydberg_circuit(num_atoms, omega, delta, interaction_v, steps, dt):
    """
    Build a QuantumCircuit simulating adiabatic preparation of a Rydberg state.
    """
    qc = QuantumCircuit()
    delta_start = -5.0

    for s in range(steps):
        t_frac = s / steps
        curr_omega = t_frac * omega
        curr_delta = delta_start + t_frac * (delta - delta_start)

        # ZZ interactions (Rydberg blockade)
        zz_theta = interaction_v * dt / 2

        # Even bonds
        for i in range(0, num_atoms - 1, 2):
            qc.cx(i, i + 1)
            qc.rz(i + 1, zz_theta)
            qc.cx(i, i + 1)

        # Odd bonds
        for i in range(1, num_atoms - 1, 2):
            qc.cx(i, i + 1)
            qc.rz(i + 1, zz_theta)
            qc.cx(i, i + 1)

        # Single-qubit gates
        for i in range(num_atoms):
            qc.rx(i, curr_omega * dt)
            z_angle = curr_delta * dt
            neighbors = (1 if i > 0 else 0) + (1 if i < num_atoms - 1 else 0)
            z_angle += neighbors * (-interaction_v * dt / 2)
            qc.rz(i, z_angle)

    return qc

print("Circuit builder defined.")

## Step 2: Order Parameter

The **Z2 staggered magnetization** measures how close the system is to the alternating $|1010\ldots\rangle$ pattern:

$$\mathcal{O} = \frac{1}{N/2} \left| \sum_i (-1)^i \langle n_i \rangle \right|$$

where $\langle n_i \rangle = (1 - \langle Z_i \rangle) / 2$.

- $\mathcal{O} \approx 0$ → **Disordered** (all atoms in ground state)
- $\mathcal{O} \to 1$ → **Z2 Ordered** (alternating excitation pattern)

In [ ]:
def calculate_order_parameter(z_expects, num_atoms):
    """Calculate Z2 staggered magnetization from ⟨Z_i⟩ values."""
    stag_mag = 0.0
    for i, z_val in enumerate(z_expects):
        n_val = (1.0 - z_val) / 2.0
        stag_mag += ((-1) ** i) * n_val
    return abs(stag_mag) / (num_atoms / 2)


def build_z_observables(n_qubits):
    """Build per-qubit Z observables: ['ZIII..', 'IZII..', ...]"""
    obs = []
    for i in range(n_qubits):
        pauli = ['I'] * n_qubits
        pauli[i] = 'Z'
        obs.append("".join(pauli))
    return obs

print("Helpers defined.")

## Step 3: Single-Point Simulation

Before sweeping the full phase diagram, let's run a single point deep in the Z2 phase to verify the simulation works.

We use Maestro's `estimate()` which computes expectation values **deterministically** — no sampling noise!

In [ ]:
# Configuration
N = 20             # Number of atoms
V = 5.0            # Interaction strength
omega = 1.5        # Rabi frequency
delta = 3.0        # Detuning (positive → favors excitation)
dt = 0.15
steps = 20
max_bond_dim = 16

# Build circuit and observables
qc = create_rydberg_circuit(N, omega, delta, V, steps, dt)
observables = build_z_observables(N)

# Run MPS simulation
t0 = time.time()
res = qc.estimate(
    observables=observables,
    simulator_type=maestro.SimulatorType.QCSim,
    simulation_type=maestro.SimulationType.MatrixProductState,
    max_bond_dimension=max_bond_dim,
)
elapsed = time.time() - t0

z_expects = res['expectation_values']
op = calculate_order_parameter(z_expects, N)

print(f"System: {N} atoms, Ω={omega}, Δ={delta}")
print(f"Order parameter: {op:.4f}")
print(f"Time: {elapsed:.2f}s")

# Visualize local densities
densities = [(1.0 - z) / 2.0 for z in z_expects]
fig, ax = plt.subplots(figsize=(10, 3))
ax.bar(range(N), densities, color=['#E74C3C' if d > 0.5 else '#3498DB' for d in densities])
ax.set_xlabel('Atom index')
ax.set_ylabel('Excitation density ⟨n_i⟩')
ax.set_title(f'Rydberg Excitation Pattern (N={N}, O={op:.3f})')
ax.set_ylim(0, 1)
ax.axhline(0.5, color='gray', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

## Step 4: Phase Diagram Sweep

Now let's sweep over a grid of $(\Delta, \Omega)$ values. At each point we compute the order parameter, building a **phase diagram** that reveals the Z2 phase transition.

> **Note:** With 64 atoms, the Hilbert space has $2^{64} \approx 1.8 \times 10^{19}$ dimensions. Statevector simulation is completely impossible — MPS makes this tractable by exploiting the 1D structure.

In [ ]:
# Phase diagram configuration
N_phase = 32         # Atoms (increase to 64 for full demo)
grid_size = 8        # Grid resolution (increase to 12 for smoother result)
V_interaction = 5.0
min_delta, max_delta = -1.0, 4.0
max_omega = 3.0
max_bond_dim = 16
dt = 0.15
steps = 20

deltas = np.linspace(min_delta, max_delta, grid_size)
omegas = np.linspace(0.1, max_omega, grid_size)
heatmap_data = np.zeros((grid_size, grid_size))
observables = build_z_observables(N_phase)

print(f"Sweeping {grid_size}×{grid_size} grid for {N_phase} atoms...")
start_time = time.time()

for i, omega in enumerate(omegas):
    for j, delta in enumerate(deltas):
        qc = create_rydberg_circuit(N_phase, omega, delta, V_interaction, steps, dt)
        try:
            res = qc.estimate(
                observables=observables,
                simulator_type=maestro.SimulatorType.QCSim,
                simulation_type=maestro.SimulationType.MatrixProductState,
                max_bond_dimension=max_bond_dim,
            )
            z_expects = res['expectation_values']
            heatmap_data[i, j] = calculate_order_parameter(z_expects, N_phase)
        except Exception:
            heatmap_data[i, j] = np.nan
    print(f"  Row {i+1}/{grid_size} done (Ω={omega:.2f})")

total_time = time.time() - start_time
print(f"Sweep completed in {total_time:.1f}s")

In [ ]:
# Plot the phase diagram
fig, ax = plt.subplots(figsize=(9, 7))
extent = [min_delta, max_delta, 0.1, max_omega]

im = ax.imshow(heatmap_data, origin='lower', extent=extent, cmap='inferno', aspect='auto')
plt.colorbar(im, ax=ax, label='Z2 Staggered Magnetization')
ax.set_xlabel(r'Detuning ($\Delta$)', fontsize=13)
ax.set_ylabel(r'Rabi Frequency ($\Omega$)', fontsize=13)
ax.set_title(f'Rydberg Atom Array Phase Diagram\n'
             f'N={N_phase} atoms, MPS χ={max_bond_dim}', fontsize=14)

ax.text(max_delta * 0.75, 0.4, 'Z2 Phase', color='white', fontsize=12, fontweight='bold', ha='center')
ax.text(min_delta + 0.5, 0.4, 'Disordered', color='white', fontsize=11, ha='center')

plt.tight_layout()
plt.show()

## Step 5: Spatial Correlations via Sampling

Now we switch from `estimate()` to `execute()` — Maestro's **sampling mode**. This produces bitstrings like a real quantum device.

### Why sampling?

The connected correlation function is:
$$C(r) = \langle n_i n_{i+r} \rangle - \langle n_i \rangle \langle n_{i+r} \rangle$$

Computing this via `estimate()` would require **O(N²)** separate observables. With sampling, we extract **all** pairwise correlations from a single set of bitstrings.

We compute the rectified version $(-1)^r C(r)$ to reveal long-range Z2 order — a flat positive line means the crystal spans the entire array.

In [ ]:
# Correlation configuration
N_corr = 15
V_corr = 10.0
target_omega = 2.0
target_delta = 5.0
T_total = 15.0
dt_corr = 0.01
steps_corr = int(T_total / dt_corr)
num_shots = 2000
max_bond_dim_corr = 32

# Build circuit and add measurements
qc = create_rydberg_circuit(N_corr, target_omega, target_delta, V_corr,
                            steps=int(steps_corr * 1.2), dt=dt_corr)
qc.measure_all()

# Sample bitstrings
print(f"Sampling {num_shots} bitstrings from MPS backend...")
t0 = time.time()
sample_res = qc.execute(
    simulator_type=maestro.SimulatorType.QCSim,
    simulation_type=maestro.SimulationType.MatrixProductState,
    shots=num_shots,
    max_bond_dimension=max_bond_dim_corr,
)
print(f"Completed in {time.time() - t0:.2f}s")
print(f"Unique bitstrings: {len(sample_res['counts'])}")

In [ ]:
# Parse bitstrings and compute correlations
counts = sample_res['counts']
parsed_samples = []
for state_str, count in counts.items():
    bits = [int(b) for b in reversed(state_str)]
    if len(bits) < N_corr:
        bits = bits + [0] * (N_corr - len(bits))
    for _ in range(count):
        parsed_samples.append(bits)

samples_matrix = np.array(parsed_samples)
densities = np.mean(samples_matrix, axis=0)

# Connected correlation function C(r)
max_r = N_corr // 2
correlations_r = np.zeros(max_r)

for r in range(1, max_r + 1):
    c_r_sum = 0.0
    terms = 0
    for i in range(N_corr - r):
        joint = np.mean(samples_matrix[:, i] * samples_matrix[:, i + r])
        product = densities[i] * densities[i + r]
        c_r_sum += (joint - product) * ((-1) ** r)
        terms += 1
    correlations_r[r - 1] = c_r_sum / terms

# Plot
fig, ax = plt.subplots(figsize=(9, 5))
r_vals = np.arange(1, max_r + 1)
ax.plot(r_vals, correlations_r, 'o-', color='#00BCD4', linewidth=2, markersize=8,
        label='Measured correlation')
ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Distance $r$ (sites)', fontsize=13)
ax.set_ylabel(r'Rectified Correlation $(-1)^r C(r)$', fontsize=13)
ax.set_title(f'Spatial Correlation Decay (N={N_corr})\n'
             f'MPS Sampling ({num_shots} shots, χ={max_bond_dim_corr})', fontsize=13)
ax.grid(True, alpha=0.2)
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

print("\nA flat positive line → long-range Z2 crystalline order.")
print("Decay towards zero → disordered phase.")

## Summary

In this notebook we demonstrated:

| Feature | Maestro API | Purpose |
|---------|-------------|----------|
| QuantumCircuit | `maestro.circuits.QuantumCircuit` | Programmatic circuit construction |
| MPS estimation | `qc.estimate(simulation_type=MPS)` | Noise-free expectation values on 64 qubits |
| MPS sampling | `qc.execute(simulation_type=MPS, shots=N)` | Bitstring sampling for correlation analysis |
| Bond dimension | `max_bond_dimension=χ` | Control accuracy vs speed tradeoff |

**Key takeaway:** MPS simulation makes 64-qubit systems tractable on a laptop. The phase diagram and correlation measurements would require $2^{64}$ amplitudes with statevector — completely impossible.